In [1]:
from tensorflow import keras
import sklearn
import IPython
import speech_recognition as sr
from pydub import AudioSegment
import subprocess
import os
import csv
import numpy as np
import pandas as pd
from tqdm import tqdm
from pydub.silence import detect_nonsilent
import librosa

In [2]:
model=keras.models.load_model('my_h5_model.h5')

In [3]:
time_list = []
audio_file = './data/공무원/공무원_경제학.wav'
def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)
#Convert wav to audio_segment
audio_segment = AudioSegment.from_wav(audio_file)
#normalize audio_segment to -20dBFS 
normalized_sound = match_target_amplitude(audio_segment, -20.0)
print("length of audio_segment={} seconds".format(len(normalized_sound)/1000))
#Print detected non-silent chunks, which in out case would be spoken words.
nonsilent_data = detect_nonsilent(normalized_sound, min_silence_len=70, silence_thresh=-32.64, seek_step=1)
#convert ms to seconds
print("start,Stop")
for chunks in nonsilent_data:
    time_list.append([chunk/1000 for chunk in chunks])

length of audio_segment=2061.839 seconds
start,Stop


In [4]:
y, sr = librosa.load('./data/공무원/공무원_경제학.wav', sr=16000)

In [5]:
def split_wav(data, sample_rate, start, end):
    start *= sample_rate
    end *= sample_rate
    return data[int(start):int(end)]

In [6]:
pad2d = lambda a, i: a[:, 0:i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0], i-a.shape[1]))))

In [7]:
np_padded_ex_mfcc_list = []
i =0
start = 0
# a=int((1100-1)/0.4)
for time in time_list :
#     y_split=split_wav(y,sr,time[0],time[1])
    if time[1]-time[0]>1 :
        y_split=split_wav(y,sr,start,time[1])[-16000:]
    else :
        y_split=split_wav(y,sr,time[0],time[1])
    mfcc = librosa.feature.mfcc(y_split, sr=16000, n_mfcc=100, n_fft=400, hop_length=160)
    mfcc = sklearn.preprocessing.scale(mfcc, axis=1)
    padded_mfcc = pad2d(mfcc, 100)
#     print(padded_mfcc.shape)
    np_padded_mfcc=np.array(padded_mfcc)
#     print(np_padded_mfcc.shape)
    np_padded_ex_mfcc=np.expand_dims(np_padded_mfcc, -1)
#     print(np_padded_ex_mfcc.shape)
    np_padded_ex_mfcc_list.append(np_padded_ex_mfcc)

C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\

C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packag

C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packag

C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packag

C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\librosa\core\spectrum.py:224: UserWarning: n_fft=400 is too small for input signal of length=368
  n_fft, y.shape[-1]
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solve

...

C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packag

C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packag

C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packag

C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packag

C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\sklearn\preprocessing\_data.py:211: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [8]:
 word_list=np.argmax(model.predict(np.array(np_padded_ex_mfcc_list)),axis = 1) 

In [9]:
start = 0
time_list2 = []
for i in range(len(word_list)) :
    if word_list[i] == 1 :
        if time_list[i][1]-start >20 :
            time_list2.append([start,time_list[i][1]])
            start =time_list[i][1]
time_list2.append([time_list2[-1][1],len(normalized_sound)/1000])

In [10]:
import speech_recognition as sr
recognizer = sr.Recognizer()
recognizer.energy_threshold = 300

#                             데이터 경로 바꿔주기
muli_audio = sr.AudioFile('./data/공무원/공무원_경제학.wav')

#                                 데이터 경로 바꿔주기
audio_seg= AudioSegment.from_wav('./data/공무원/공무원_경제학.wav')
total_in_ms= len(audio_seg)

Text_list = []
Text_list2 = []
for start,end in tqdm(time_list2) : #tqdm(range (1,int(total_in_ms/1000/30)+1)) :
    with muli_audio as source:
        s_audio = recognizer.record(source,offset = start,duration=(end-start))
    Text_list.append(recognizer.recognize_google(audio_data=s_audio, language="ko-KR",show_all=True))

100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [05:17<00:00,  3.45s/it]


In [11]:
last_list = []
for i in Text_list :
    print(i['alternative'][0]['transcript'])
    print()
    last_list.append(i['alternative'][0]['transcript'])

제 1장 경제기초 시작하겠습니다 제일 장은 독립적으로 시험에 출제 된다기보다는 경제공부를함에 있어서 반드시 필요한 기초개념과 용어들이 나와 있기 때문에 이런 내용들이 있구나 가볍게 이해하시고 넘어 가시면 되겠습니다 첫 번째 논점 정지하게 기본적 개념입니다 만약에 우리한테

한 포대 있다고 칩시다 밀가루로 맛있는 빵을 만들려고하는데요 사람들이 빵을 좋아하기 때문에 만약에 우리가 빵을 무한히 많이 만들 수 있다면 많은 사람들이 아주 행복하게 빵을 소비할 수 있을 것입니다 하지만 현실적으로 빵을 무한히 많이 만드는 것은 불가능한데요 왜냐하면 빵을 만드는 재료 밀가루

제한되어 있기 때문입니다 이처럼 자원의 부족함 자원의 희소성으로 인해서 인간의 욕망을 충분히 충족시키지 못하는 문제가 발생하게 되는데 우리 이러한 문제를 경제 문제라고 합니다 경제문제가 심각할수록 인간은 물질적으로 충분히 행복하지 못한 문제가 발생하게 되기 때문에

정재 문제를 해결하기 위한 학문이 바로 경제학이다라고 정할 수가 있습니다 정지 하거든요 인간의 욕망을 차에 대한 충족시키기 위해서 완벽하게 충족시킬 수는 없겠지요 왜냐면 자원의 유한 하기 때문에 하지만 최대한 충족시키기 위해서 부족한 자원 희소한 자원을 합리적으로 가장 효율적으로

가장 경제적으로 활용하는 방법을 연구하는 학문을 경제학이라고 할 수가 있겠습니다 즉 경제학은 경제 문제를 해결하기 위한 학문이다 주어진 자원을 최대한 합리적 효율적 경제적으로 활용해서 각종 경제 문제를 해결하는 방법을 연구하는 모색하는 학문

라고 정리 할 수가 있겠습니다 경제문제는 크게 3가지 문제로 대별 됩니다 첫 번째 어떤 재활을 얼마나 생산 할 것인가 우리가 밀가루로 빵을 생산한다고 썼을 때 빵 두 종류가 다양하게 식빵도 있고요 소보로빵도 있고 단팥빵도 있습니다 이런 식으로 주어진 밀가루의 양은 1 4 0 제한되어 있는데이 한 포대 밀가루로 속으로 방은 몇 개를 만들고 식빵은 몇 개를

각 방에 종류마다 몇 개씩 만들것인지 생산물의 종류 수량결정 문제가 첫

In [12]:
df=pd.DataFrame(time_list2,columns=['start_time','end_time'])

In [13]:
df["script"]=last_list

In [14]:
df.to_csv("경제학_방법1.csv",encoding="utf-8-sig",index=False)

In [16]:
from konlpy.tag import Kkma
import pandas as pd

kkm = Kkma()

sentence_list = []
for script in last_list: 
    sentence_list.append(kkm.sentences(script))

script_list=[]
script2_list=[]
new_sentence_list = []
noun_list = []
pos_last_list = []
for sentences in sentence_list :
    script =[]
    for st in sentences[-1:] :
        pos=kkm.pos(st)
        if (pos[-1][1] =='EFN') | (pos[-1][1] =='EFA'):
            script.append(st +". ")
            script2_list.append(st +". ")
        elif pos[-1][1] =='EFQ':  
            script.append(st +"? ")
            script2_list.append(st +"? ")
        elif (pos[-1][1] =='EFI') | (pos[-1][1] =='EFO') :   
            script.append(st +"! ")
            script2_list.append(st +"! ")
        else :
            script.append(st +" ")
            script2_list.append(st +" ")            
        for ps in pos :
            if ps[1][:2] == "NN" :
                noun_list.append(ps[0])
        pos_last_list.append(pos[-1][1])
    script_list.append(script)

pd.DataFrame(pos_last_list,columns=["pos"])["pos"].value_counts()

NNG    21
JKO     9
EFN     9
ECD     8
ETD     5
ECS     5
JKS     5
JKM     4
ECE     4
JX      4
ETN     3
NNB     2
VV      2
EFQ     2
JKG     2
NNM     1
NR      1
MAG     1
JC      1
NNP     1
NP      1
VCN     1
Name: pos, dtype: int64

In [17]:
data=pd.DataFrame(pos_last_list,columns=["pos"])

In [18]:
end_word_cnt = 0;
not_end_word_cnt = 0;
size=data["pos"].size
for i in data["pos"] :
    if i[0:2]=='EF' :
        end_word_cnt += 1;
    else : 
        not_end_word_cnt += 1;
print(f"종결어미 : {end_word_cnt/size} 종결어미 X: {not_end_word_cnt/size} ")

종결어미 : 0.11956521739130435 종결어미 X: 0.8804347826086957 
